In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from bs4 import BeautifulSoup
import torchvision
from torchvision import transforms, datasets, models
import torch
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import matplotlib.patches as patches
import os
from importlib import reload
import cvn_utils
import kaggle_utils
import math

In [2]:
!nvidia-smi

Sun Sep 13 01:19:46 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.01    Driver Version: 418.87.01    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
torch.cuda.is_available()

True

## Find image file from label file

In [4]:
def is_kaggle_annotation_file(file_name):
    return 'annotations' in file_name and 'xml' in file_name
 
def is_cvn_json_file(file_name):
    return ('faces' in file_name) and ('json' in file_name)

def cvn_json_has_box(json_fn):
    processor = cvn_utils.JsonFileProcessor(json_fn)
    processor.load()
    return len(processor.resized_boxes) != 0
        

### Walk over annotation files of both kaggle and cvn

In [5]:
reload(cvn_utils)
kaggle_label_fns = []
cvn_label_fns = []
for dirname, _, filenames in os.walk('/home/yangxu/face_mask_detection_workspace/'):
    for filename in filenames:
        annotation_full_name = os.path.join(dirname, filename)
        if is_kaggle_annotation_file(annotation_full_name):
            kaggle_label_fns.append(annotation_full_name)
        elif is_cvn_json_file(annotation_full_name):              
            if cvn_json_has_box(annotation_full_name):
                cvn_label_fns.append(annotation_full_name)
            else:
                continue
        else:
            continue        
           
print ('kaggle file num = {}, cvn file num = {}, all label file num = {}'.format(
        len(kaggle_label_fns), 
        len(cvn_label_fns),
        len(kaggle_label_fns) + len(cvn_label_fns)))


kaggle file num = 853, cvn file num = 517, all label file num = 1370


In [6]:
class MaskDataset(object):
    def __init__(self, transforms):
        self.transforms = transforms        
#         self.all_label_fns = kaggle_and_cvn_label_fns
        self.all_label_fns = cvn_label_fns

    def gen_target_kaggle(self, idx, label_fn):       
        return kaggle_utils.generate_target(idx, label_fn)
        
    def __getitem__(self, idx):        
        img = None
        taret = None
        
        label_fn = self.all_label_fns[idx]        
        
        #Generate Label
        if is_kaggle_annotation_file(label_fn):
            img_fn = kaggle_utils.find_image_file_kaggle(label_fn)
            img = Image.open(img_fn).convert("RGB") 
            target = kaggle_utils.generate_target(idx, label_fn)
            if self.transforms is not None:
                img = self.transforms(img)
#                 print('kaggle image shape = {}, min = {}, max = {}'.format( 
#                       img.shape, torch.min(img), torch.max(img)))
        else:
            processor = cvn_utils.JsonFileProcessor(label_fn)
            processor.load()
            img = processor.generate_image_tensor()
            target = processor.generate_target(idx)
#             cvn_idx_to_label_fn_dict[idx] = (img, target, label_fn)
#             print('cvn image shape = {}, min = {}, max = {}'.format( 
#                       img.shape, torch.min(img), torch.max(img)))
        
        

        return img, target

    def __len__(self):
        return len(self.all_label_fns)

In [7]:
data_transform = transforms.Compose([
        transforms.ToTensor(), 
    ])

In [8]:
def collate_fn(batch):
    return tuple(zip(*batch))

dataset = MaskDataset(data_transform)
data_loader = torch.utils.data.DataLoader(
 dataset, batch_size=4, collate_fn=collate_fn, num_workers=1)


In [9]:
def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

In [10]:
model = get_model_instance_segmentation(3)

In [11]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# for imgs, annotations in data_loader:
#     imgs = list(img.to(device) for img in imgs)   
#     annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
   
#     break

In [ ]:
num_epochs = 25
model.to(device)
    
# parameters
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)

len_dataloader = len(data_loader)

for epoch in range(num_epochs):
    model.train()
    i = 0    
    epoch_loss = 0
    for imgs, annotations in data_loader:
        i += 1
        
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
               
        loss_dict = model(imgs, annotations)
        losses = sum(loss for loss in loss_dict.values())      
            
        optimizer.zero_grad()
        losses.backward()
        optimizer.step() 
        epoch_loss += losses
        if i % 10 == 0:
            print('== iteration loss: {}, {}'.format(i, losses))
    print('==== epoch = {}, epoch_loss = {}'.format(epoch, epoch_loss))


/home/yangxu/.local/lib/python3.5/site-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


== iteration loss: 10, 0.4195733964443207
== iteration loss: 20, 0.19084376096725464
== iteration loss: 30, 0.22622358798980713
== iteration loss: 40, 0.27754709124565125
== iteration loss: 50, 0.15347209572792053
== iteration loss: 60, 0.14339622855186462
== iteration loss: 70, 0.19514775276184082
== iteration loss: 80, 0.25687921047210693
== iteration loss: 90, 0.2749450206756592
== iteration loss: 100, 0.2593366801738739
== iteration loss: 110, 0.11658702790737152
== iteration loss: 120, 0.08648967742919922
== iteration loss: 130, 0.0751197338104248
==== epoch = 0, epoch_loss = 29.216768264770508
== iteration loss: 10, 0.3130526542663574
== iteration loss: 20, 0.14033275842666626
== iteration loss: 30, 0.21675266325473785
== iteration loss: 40, 0.1584397256374359
== iteration loss: 50, 0.08838857710361481
== iteration loss: 60, 0.11065880954265594
== iteration loss: 70, 0.13527792692184448
== iteration loss: 80, 0.21219906210899353
== iteration loss: 90, 0.21110881865024567
== itera

== iteration loss: 70, 0.01818263903260231
== iteration loss: 80, 0.06691490113735199
== iteration loss: 90, 0.0729772299528122
== iteration loss: 100, 0.05982150137424469
== iteration loss: 110, 0.019115695729851723
== iteration loss: 120, 0.02370421774685383
== iteration loss: 130, 0.010228545404970646
==== epoch = 13, epoch_loss = 5.758670330047607
== iteration loss: 10, 0.09413512051105499
== iteration loss: 20, 0.037096358835697174
== iteration loss: 30, 0.06340176612138748
== iteration loss: 40, 0.038266416639089584
== iteration loss: 50, 0.016228891909122467
== iteration loss: 60, 0.016429532319307327
== iteration loss: 70, 0.02789754420518875
== iteration loss: 80, 0.08571842312812805
== iteration loss: 90, 0.09493854641914368
== iteration loss: 100, 0.06605976819992065
== iteration loss: 110, 0.0214392002671957
== iteration loss: 120, 0.021621311083436012
== iteration loss: 130, 0.012854911386966705
==== epoch = 14, epoch_loss = 6.051639080047607
== iteration loss: 10, 0.10699

In [ ]:
torch.save(model.state_dict(),'model.pt')

In [ ]:
def plot_image(img_tensor, target_tensor):
    fig,ax = plt.subplots(1)
    img = img_tensor.cpu().data

    # Display the image
    ax.imshow(img.permute(1, 2, 0))

    for box in target_tensor['boxes'].detach().cpu().numpy():
        xmin, ymin, xmax, ymax = box
        rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='r',facecolor='none')

        # Add the patch to the Axes
        ax.add_patch(rect)

    plt.show()    

In [ ]:
plot_image(nan_img, nan_label)

In [ ]:
nan_label['boxes']

In [ ]:
nan_label_before_device['boxes']

In [ ]:
plot_image(nan_img_before_device, nan_label_before_device)


In [ ]:
nan_label_before_device['boxes']

In [ ]:
device_nan_label_before_device = nan_label_before_device['boxes'].to(device)

In [ ]:
device_nan_label_before_device = {k: v.to(device) for k, v in nan_label_before_device.items()}

In [ ]:
nan_label_before_device

In [ ]:
plot_image(nan_img_before_device, device_nan_label_before_device)

In [ ]:
plot_image(nan_img_before_model, nan_label_before_model)

In [ ]:
nan_label_before_model

In [ ]:
nan_label['boxes']

In [ ]:
for imgs, annotations in data_loader:    
    imgs = list(img.to(device) for img in imgs)
    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    plot_image(imgs[0], annotations[0])
    break

In [ ]:
len(imgs)

In [ ]:
for i in range(4):
    plot_image(nan_img_batch[i], nan_label_batch[i])

In [ ]:
nan_label_batch[0]

In [ ]:
cvn_idx_to_label_fn_dict[180]